In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train_ver2.csv')
test = pd.read_csv('test_ver2.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#제품이름
prods = train.columns[24:].tolist()

In [4]:
train[prods].isnull().sum()

ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
ind_reca_fin_ult1        0
ind_tjcr_fin_ult1        0
ind_valo_fin_ult1        0
ind_viv_fin_ult1         0
ind_nomina_ult1      16063
ind_nom_pens_ult1    16063
ind_recibo_ult1          0
dtype: int64

In [4]:
#제품 변수 결측치 0으로
train[prods] = train[prods].fillna(0.0).astype(np.int8)

In [5]:
#24개 제품 중 하나도 보유하지 않은 고객데이터 제거
no_product = train[prods].sum(axis=1) == 0
train = train[~no_product]
del no_product

In [6]:
for col in prods :
    test[col] = 0 #일단 test 데이터에 target을 0으로 initialize
df = pd.concat([train,test], axis = 0)

In [7]:
del train
del test

In [8]:
df.shape

(12020685, 48)

In [8]:
#학습에 필요한 변수를 담는 list
features = []

## categorical variables

In [9]:
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols :
    df[col], _ = df[col].factorize(na_sentinel =-99) #범주형 변수들의 factor값들을 숫자로
    
features+=categorical_cols

In [11]:
df[categorical_cols].head()

,ind_empleado,pais_residencia,sexo,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,nomprov,segmento
0,0,0,0,0,0,0,-99,0,0,0,0,0
1,0,0,1,1,0,1,-99,1,0,0,1,1
2,0,0,1,1,0,0,-99,1,0,0,1,1
3,0,0,0,1,0,0,-99,2,0,0,2,1
4,0,0,1,0,0,0,-99,1,0,0,2,1


## numeric variables

In [10]:
#NA값들 -99로 바꾸고 int 타입으로 변환
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True) #중간에 p라는 값이 있음
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [11]:
# 학습에 사용할 수치형 변수를 features에 add
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

## feature engineering

In [22]:
#fecha_alta : 고객이 첫 게약을 맺은 날짜
#ult_fec_cli_1t : 고객이 마지막으로 1등금이었던 날짜
df[['fecha_alta', 'ult_fec_cli_1t']].head()

,fecha_alta,ult_fec_cli_1t
0,2015-01-12,NaN
1,2012-08-10,NaN
2,2012-08-10,NaN
3,2012-08-10,NaN
4,2012-08-10,NaN


In [29]:
df['fecha_alta'].isnull().sum()

21866

In [13]:
for x in df['fecha_alta'][:5000] :
    if x.__class__ is float : #x.__class__ == type(x)
        print(x)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


##### year, month

In [12]:
#NA는 0으로 하고, 월과 년을 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1]))
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0]))

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1]))
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0]))

features += ['fecha_alta_month', 'fecha_alta_year','ult_fec_cli_1t_month','ult_fec_cli_1t_year']

In [13]:
#그 외 NA는 -99로
df.fillna(-99, inplace = True)

##### lag-1

In [14]:
#2015-01-25 - >1 / 2016-06-28 -> 18
def date_to_int(str_date) :
    Y,M,D = [int(a) for a in str_date.strip().split('-')]
    int_date = (int(Y) - 2015) * 12 +int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [15]:
print('# of id : ', len(df['ncodpers'].unique()), '#length of data', len(df) )

('# of id : ', 945515, '#length of data', 12020685)


In [18]:
#memory error
'''
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
#ncodpers : 고객 고유 번호
df_lag['int_date'] += 1
#int_date에 1씩 추가했기 때문에, 저번 달의 제품 정보가 삽입
df_tran = df.merge(df_lag, on = ['ncodpers', 'int_date'], how = 'left')
'''

In [20]:
df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_alta_month,fecha_alta_year,ult_fec_cli_1t_month,ult_fec_cli_1t_year,int_date
0,2015-01-28,1375586,0,0,0,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,1.0,2015.0,0.0,0.0,1
1,2015-01-28,1050611,0,0,1,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,8.0,2012.0,0.0,0.0,1
2,2015-01-28,1050612,0,0,1,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,8.0,2012.0,0.0,0.0,1
3,2015-01-28,1050613,0,0,0,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,8.0,2012.0,0.0,0.0,1
4,2015-01-28,1050614,0,0,1,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,8.0,2012.0,0.0,0.0,1


In [36]:
df2 = df.groupby(['ncodpers'])['int_date'].agg(['min','max','size'])
df2.columns = ['lag_min','lag_max','lag_size']
df2['my'] = df2['lag_max'] - df2['lag_min'] + 1

In [47]:
print('true : ', sum(df2['my'] == df2['lag_size']), 'df2 size : ' ,len(df2))

('true : ', 918066, 'df2 size : ', 945515)


##### ncodpers끼리 불러서 해보자

In [73]:
mycol = [col+'_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
for col in mycol :
    df[col] = np.nan

In [61]:
df2 = df.loc[df.ncodpers == 15942]
my_lag = df2['int_date']
my_lag

2933762     5
3565698     6
618348     18
Name: int_date, dtype: int8

In [70]:
for lag in my_lag :
    try :
        print(list(df2.loc[df2['int_date'] == lag, 'int_date'])[0])
        print(list(df2.loc[df2['int_date'] == lag-1, 'int_date'])[0])
        
    except IndexError :
        print('')

5

6
5
18



### split df

In [17]:
df = df.sort_values(['ncodpers','int_date'])
df.reset_index(drop = True, inplace=True)

In [18]:
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
#ncodpers : 고객 고유 번호
df_lag['int_date'] += 1
#int_date에 1씩 추가했기 때문에, 저번 달의 제품 정보가 삽입

In [23]:
df.shape

(12020685, 53)

In [25]:
df_lag.shape

(12020685, 53)

In [28]:
len(df.ncodpers.unique())

945515

In [30]:
945515/5

189103

In [26]:
my_ind = [0, 189103, 378206, 567309, 756412, 945515]

In [35]:
for i in range(len(my_ind)-1) :
    ind1 = df.loc[df.ncodpers == df.ncodpers.unique()[my_ind[i]:][0]].index[0]
    ind2 = df.loc[df.ncodpers == df.ncodpers.unique()[:my_ind[i+1]][-1]].index[-1]
    
    print(i+1)
    df.iloc[ind1:ind2].to_csv('df_{}.csv'.format(i+1))
    df_lag.iloc[ind1:ind2].to_csv('df_lag_{}.csv'.format(i+1))

1
2
3
4
5
